# Agregación y agrupación

Una pieza esencial en el análisis de grandes datos es un resumen eficiente: calcular agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()``, y ``max()``, en las que un único número da una idea de la naturaleza de un conjunto de datos potencialmente grande.
En esta sección, exploraremos las agregaciones en Pandas, desde operaciones simples similares a las que hemos visto en los arrays de NumPy, hasta operaciones más sofisticadas basadas en el concepto de ``groupby``.

Por comodidad, utilizaremos la misma función mágica ``display`` que hemos visto en secciones anteriores:

In [11]:
import numpy as np
import pandas as pd

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

> - **CÓMO VER LA VERSIÓN DE PANDAS:**

In [ ]:
pd.__version__

## Datos de Planetas

Aquí usaremos el conjunto de datos Planetas, disponible a través del [paquete Seaborn](http://seaborn.pydata.org/) (ver [Visualization With Seaborn](14_Visualization-With-Seaborn.ipynb)).
Proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como *planetas extrasolares* o *exoplanetas* para abreviar). Puede descargarse con un simple comando Seaborn:

In [46]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [47]:
planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [6]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Contiene algunos detalles sobre los más de 1.000 planetas extrasolares descubiertos hasta 2014.

>> ## Agregación simple en Pandas

> #### **AGREGACIÓN EN SERIES**

Anteriormente, hemos explorado algunas de las agregaciones de datos disponibles para arrays NumPy (["Agregaciones: Mín, Máx, y todo lo que hay en medio"](4_Computation-on-arrays-aggregates.ipynb)).
Al igual que con un array unidimensional NumPy, para una ``Serie`` Pandas los agregados devuelven un único valor:

In [14]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [15]:
ser.sum()

2.811925491708157

In [21]:
ser.mean()

0.5623850983416314

In [23]:
round(ser.mean(),2)

0.56

> #### **AGREGACIÓN EN DATAFRAMES**

Por defecto los agregados **devuelven resultados** dentro de **cada columna**:

In [24]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


> #### **`mean()` por columnas**

Devuelve la media de cada columna, ya que Pandas siempre trabaja por columnas a menos que le especifique lo contrario: 

In [27]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

> #### **`mean()` por columna en particular**

Si quiero solo la de la columna 'A' debo especificarlo. 

**Hay 2 formas:**
1.  **`df['A'].mean()`** > Devuelve el número de la media de esa columna solamente 
2.  **`df[['A']].mean()`**> DOBLE CORCHETES: Devuelve resultado de la media de la columna, nombre de la misma y dtype

In [28]:
df['A'].mean()

0.4778883735637184

In [29]:
df[['A']].mean()

A    0.477888
dtype: float64

> #### **`mean()` por filas con `axis='columns'`**

Especificando el argumento ``axis``, puede agregar dentro de cada fila:

In [30]:
df.mean(axis='columns') # Da la media de cada fila

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [31]:
df.mean(axis='rows') # Da la media de cada columna. ESTO VIENE POR DEFECTO > NO hace falta ponerlo

A    0.477888
B    0.443420
dtype: float64

> #### **`.describe()`**

Es un método que calcula varios agregados comunes para cada columna y devuelve el resultado.
 
- Si se ejecuta `dataframe.describe()` sin especificar `(include=)`, el resultado mostrará únicamente estadísticas para las **columnas numéricas del DataFrame.**

Usemos esto en los datos de Planetas, por ahora eliminando las filas con valores perdidos con `.dropna()`:

In [34]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


> **`.describe(include=)` + `'all'` y `object`**

Al utilizar `include='all'`, se solicita que se incluyan tanto **columnas numéricas** como **columnas categóricas o de tipo objeto** en la descripción.

In [35]:
planets.dropna().describe(include='all')

,method,number,orbital_period,mass,distance,year
count,498,498.00000,498.000000,498.000000,498.000000,498.000000
unique,2,NaN,NaN,NaN,NaN,NaN
top,Radial Velocity,NaN,NaN,NaN,NaN,NaN
freq,497,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.73494,835.778671,2.509320,52.068213,2007.377510
std,NaN,1.17572,1469.128259,3.636274,46.596041,4.167284
min,NaN,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,NaN,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,NaN,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,NaN,2.00000,999.600000,2.867500,59.332500,2011.000000


In [36]:
planets.dropna().describe(include=object)

,method
count,498
unique,2
top,Radial Velocity
freq,497


Esta puede ser una forma útil de empezar a comprender las propiedades generales de un conjunto de datos.
Por ejemplo, en la columna ``year`` vemos que, aunque ya en 1989 se descubrieron exoplanetas, la mitad de los conocidos no se descubrieron hasta 2010 o después.
Esto se debe en gran parte a la misión *Kepler*, un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

>> ### **AGREGACIONES**

La siguiente tabla resume algunas otras agregaciones incorporadas en Pandas:

| Agregacion               | Descripcion                     |
|--------------------------|---------------------------------|
| ``count()``              | Número total de artículos       |
| ``first()``, ``last()``  | Primer y último punto           |
| ``mean()``, ``median()`` | Media y mediana                 |
| ``min()``, ``max()``     | Mínimo y máximo                 |
| ``std()``, ``var()``     | Desviación estandar y varianza  |
| ``mad()``                | Desviación media absoluta       |
| ``prod()``               | Producto de todos los artículos |
| ``sum()``                | Suma de todas las partidas      |

Todos estos son métodos de los objetos ``DataFrame`` y ``Series``.

Sin embargo, para profundizar en los datos, los agregados simples no suelen ser suficientes.
El siguiente nivel de integración de datos es la operación ``groupby``, que permite calcular rápida y eficazmente agregados sobre subconjuntos de datos.

>>> ## **GroupBy: Dividir, Aplicar, Combinar**
Se utiliza cuando **los datos ya están limpios y ordenados** > Ya que si quiero hacerlo con datos sucios y saco la media, o la suma de un DF, no serán resultados correctos.

Las agregaciones simples pueden darle una idea de su conjunto de datos, pero a menudo preferiríamos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en la llamada operación ``groupby``.
El nombre "group by" procede de un comando del lenguaje de bases de datos SQL, pero quizá sea más ilustrativo pensar en él en los términos acuñados por Hadley Wickham, famoso por Rstats: *dividir, aplicar, combinar*.

>> ### **DIVIDIR, APLICAR Y COMBINAR = Split, apply, combine**

Un ejemplo canónico de esta operación dividir-aplicar-combinar, donde el "aplicar" es una agregación sumatoria.

Esto aclara lo que consigue el ``groupby``:

- El paso **split** consiste en dividir y agrupar un ``DataFrame`` en función del valor de la clave especificada.
- El paso **apply** implica calcular alguna función, normalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso **combinar** combina los resultados de estas operaciones en una matriz de salida.

Aunque esto podría hacerse manualmente utilizando alguna combinación de los comandos de enmascaramiento, agregación y fusión descritos anteriormente, es importante tener en cuenta que *las divisiones intermedias NO necesitan instanciarse explícitamente*, es decir, crear variables. 

En su lugar, ``GroupBy`` puede (a menudo) hacer esto en una sola pasada sobre los datos, actualizando la suma, media, recuento, min, u otro agregado para cada grupo a lo largo del camino.
El poder del ``GroupBy`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo bajo el capó, sino que piensa en la *operación en su conjunto*.

Como ejemplo concreto, echemos un vistazo al uso de Pandas para el cálculo mostrado en este diagrama.
Empezaremos creando el ``DataFrame`` de entrada:

In [10]:
import pandas as pd

In [37]:
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'VV': range(6)})
df

,department,VV
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de dividir-aplicar-combinar puede calcularse con el método ``groupby()`` de ``DataFrame``s, pasando el **NOMBRE DE LA COLUMNA CLAVE DESEADA**:

In [38]:
df.groupby('department')

Observa que lo que se devuelve no es un conjunto de ``DataFrame``, sino un OBJETO ``DataFrameGroupBy``.

Este objeto es donde está la magia: se puede pensar en él como una vista especial del ``DataFrame``, que **está preparado para profundizar en los grupos, pero no hace ningún cálculo real hasta que se aplica la agregación.**
Este enfoque de "evaluación perezosa" significa que esa agrupación la guarda en una memoria para luego poder operar con las agregaciones rápidamente. 

> **`as_index=False`** 

Se utiliza para que no tome la columna (que le paso como parámetro para agrupar) como índice, sino que la deje como columna y cree otro índice natural para esa agrupación. 

In [41]:
df_grouped = df.groupby('department', as_index=False).max()
df_grouped

,department,VV
0,A,3
1,B,4
2,C,5


>>> ## **El objeto GroupBy**
El objeto ``GroupBy`` es una abstracción muy flexible.
En muchos sentidos, puedes simplemente tratarlo como si fuera una colección de ``DataFrame``s, y hace las cosas difíciles bajo el capó. Veamos algunos ejemplos utilizando los datos de Los Planetas.

Quizás las operaciones más importantes disponibles en un ``GroupBy`` son *agregar*, *filtrar*, *transformar* y *aplicar*.

>> #### **Indexación de columnas**

El objeto ``GroupBy`` soporta la indexación por columnas de la misma forma que el ``DataFrame``, y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo

In [46]:
planets.groupby('method')

In [47]:
planets.groupby('method')['orbital_period']

> #### **`.count()`**

Devuelve el DF con todo el conteo de las demás columnas, segun cada método, o sea, cada valor de la columna `'method'`

In [48]:
planets.groupby('method').count()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,2,2,0,2,2
Eclipse Timing Variations,9,9,2,4,9
Imaging,38,12,0,32,38
Microlensing,23,7,0,10,23
Orbital Brightness Modulation,3,3,0,2,3
Pulsar Timing,5,5,0,1,5
Pulsation Timing Variations,1,1,0,0,1
Radial Velocity,553,553,510,530,553
Transit,397,397,1,224,397


* Si quiero ver solo una columna (`['orbital_period']`) en relación a la columna por la cual se hizo la agrupación (`.groupby('method')`), la especifico y pongo el `.count()` a lo último.
  
Con **doble corchetes [[ ]]** lo muestra al estilo DF, y con **un solo corchete [ ]** lo muestra en formato serie. 

In [60]:
planets.groupby('method', as_index=False)[['orbital_period']].count()

,method,orbital_period
0,Astrometry,2
1,Eclipse Timing Variations,9
2,Imaging,12
3,Microlensing,7
4,Orbital Brightness Modulation,3
5,Pulsar Timing,5
6,Pulsation Timing Variations,1
7,Radial Velocity,553
8,Transit,397
9,Transit Timing Variations,3


Aquí hemos seleccionado un grupo ``Series`` particular del grupo original ``DataFrame`` por referencia a su nombre de columna.

Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [50]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


> #### **`.unique()` y `nunique()`**

- `.unique()` > Devuelve en un array todos los valores únicos de la columna `'method'`

`len(planets['method'].unique())` > Con `len()` devuelve el número de valores únicos. Que es lo mismo que hacer: 

- `nunique()` > devuelve el número de valores únicos.


> `.unique()`

In [55]:
planets['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

In [53]:
len(planets['method'].unique())

10

> `nunique()`

In [54]:
planets['method'].nunique()

10

> #### **`.mean()`**

In [58]:
planets.groupby('method', as_index=False)['orbital_period'].mean()

,method,orbital_period
0,Astrometry,631.180000
1,Eclipse Timing Variations,4751.644444
2,Imaging,118247.737500
3,Microlensing,3153.571429
4,Orbital Brightness Modulation,0.709307
5,Pulsar Timing,7343.021201
6,Pulsation Timing Variations,1170.000000
7,Radial Velocity,823.354680
8,Transit,21.102073
9,Transit Timing Variations,79.783500


* Lo anterior, se puede hacer con una **MÁSCARA BOOLEANA** y averiguar solamente la media de la fila 0 - 'Astrometry':

In [62]:
planets[planets['method']=='Astrometry'][['orbital_period']].mean()

orbital_period    631.18
dtype: float64

Esto da una idea de la escala general de periodos orbitales (en días) a la que es sensible cada método.

>> #### **Iteración sobre grupos**

El objeto ``GroupBy`` permite la iteración directa sobre los grupos, devolviendo cada grupo como una ``Series`` o ``DataFrame``.

El siguiente ejemplo devuelve: 
- Cada **MÉTODO** que hay en la columna/índice 'method'
- Y la **LONGITUD** de cada uno en forma de tuplas = (cantidad de filas, cantidad de columnas)

In [21]:
for (method, group) in planets.groupby('method'): 
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Esto puede ser útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido utilizar la funcionalidad incorporada ``apply``, que discutiremos momentáneamente.

>> #### **Métodos de envío**

A través de la magia de las clases de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será pasado y llamado en los grupos, ya sean objetos ``DataFrame`` o ``Series``.
Por ejemplo, puedes utilizar el método ``describe()`` de ``DataFrame`` para realizar un conjunto de agregaciones que describan cada grupo en los datos.

> `.describe()` + `.unstack()` 

Poniendole esto, lo devuelve en forma de serie y es más fácil acceder a los datos. Porque primero nombro count o max y luego el nombre del método.
- De lo contrario, SIN el .unstack() > Me lo devuelve como DF, y tengo que acceder por la columna, luego poner el nombre del índice ('method') y luego, el índice (nombre del método)

In [63]:
planets.groupby('method')['year'].describe()


,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


Observar esta tabla nos ayuda a entender mejor los datos: por ejemplo, la gran mayoría de los planetas se han descubierto por los métodos de Velocidad Radial y Tránsito, aunque este último no se hizo común (debido a nuevos telescopios más precisos) hasta la última década.
Los métodos más recientes parecen ser el de la Variación Temporal del Tránsito y el de la Modulación del Brillo Orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Este es solo un ejemplo de la utilidad de los métodos de envío.
Fíjate en que se aplican *a cada grupo individual*, y los resultados se combinan dentro de ``GroupBy`` y se devuelven.
De nuevo, cualquier método válido de ``DataFrame`` o ``Series`` puede utilizarse en el objeto ``GroupBy`` correspondiente, lo que permite realizar operaciones muy flexibles y potentes.

>> ## **Agregar, filtrar, transformar, aplicar**

**``GroupBy``** **+**
- ``aggregate()``
- ``filter()``
- ``transform()``
- ``apply()``

La discusión anterior se centró en la agregación para la operación combinar, pero hay más opciones disponibles.
En particular, los objetos ``GroupBy`` tienen métodos ``aggregate()``, ``filter()``, ``transform()``, y ``apply()`` que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

Para el propósito de las siguientes subsecciones, utilizaremos este ``DataFrame``: 

In [2]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


> ### **Agregación = ``.aggregate()`` o `.agg()`**

- **Realiza varias agregaciones a la vez** 
- Pueden ponerse también funciones internas de Python como `sum` o hasta de Numpy como `np.mean`
- Puede tomar una cadena, una función o una lista de agregaciones y calcular todos los agregados a la vez.
- También pueden aplicarse agregaciones a cada columna en particular en forma de diccionario.


He aquí un ejemplo rápido que combina todos estos métodos:

In [4]:
df.groupby('key').aggregate(['min', 'median', 'max', np.mean, sum])

C:\Users\Ximena\AppData\Local\Temp\ipykernel_7948\3655039847.py:1: FutureWarning: The provided callable <function mean at 0x000002029756E160> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby('key').aggregate(['min', 'median', 'max', np.mean, sum])
C:\Users\Ximena\AppData\Local\Temp\ipykernel_7948\3655039847.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.groupby('key').aggregate(['min', 'median', 'max', np.mean, sum])


data1                     data2                    
      min median max mean sum   min median max mean sum
key                                                    
A       0    1.5   3  1.5   3     3    4.0   5  4.0   8
B       1    2.5   4  2.5   5     0    3.5   7  3.5   7
C       2    3.5   5  3.5   7     3    6.0   9  6.0  12

Otro patrón útil es pasar un **diccionario** que asigne los **nombres de las columnas** a las **agregaciones** que deben aplicarse a esa columna:

In [5]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


- Si quiero **aplicarle a una columna más de una agregación**, tengo que pasarlas en forma de **LISTA**, ya que si pongo dos veces el nombre de la columna, RECORDAR, que en los **diccionarios** las **claves son únicas**, entonces la última reemplazaría la primera. 

In [6]:
df.groupby('key').aggregate({'data1': ['min', 'mean'],
                             'data2': 'max'})

data1      data2
      min mean   max
key                 
A       0  1.5     5
B       1  2.5     7
C       2  3.5     9

> ### **Filtrado = ``.filter( )``**

Una operación de filtrado permite descartar datos en función de las propiedades del grupo.

- Si quiero que se guarden estas operaciones con agregaciones, debo **guardarlas en una variable** > Si no, solo lo muestra!

Por ejemplo, creo una **función** donde digo: devolvelme `x` de `'data2'` si el mínimo de esa columna es mayor (>) que 1:

In [7]:
def filter_func(x):
    return x['data2'].min() > 1

In [8]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [9]:
filter_func(df) # Esto devuelve False o True. Por eso hay que hacerlo por groupby()

False

In [12]:
display('df', "df.groupby('key').min()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').min()
     data1  data2
key              
A        0      3
B        1      0
C        2      3

df.groupby('key').filter(filter_func)
  key  data1  data2
0   A      0      5
2   C      2      3
3   A      3      3
5   C      5      9

NO devuelve la 'fila/key B' en el filtrado, porque no cumplía con la condición en `'data2'` de ser mayor que 0, entonces la descarta. <br>
Y tampoco devuelve el 7, porque yo estaba agrupado por key (B) y la función dice que el mínimo debe ser mayor que 0. O sea, que el mínimo de 'B' sería 0, por eso la descarta. 

In [14]:
df[df['data2'] > 1]

,key,data1,data2
0,A,0,5
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [15]:
df # NO se guardan las operaciones con las agregaciones. Deben guardarse en una VARIABLE!

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


> ### **Transformación = `.transform()`**

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinar.
Para tal transformación, la salida tiene la misma forma que la entrada.

Por ejemplo, voy a transformar los datos, restando a cada elemento la media del grupo agrupado por 'key' que sería, la media de 'data1' de A, de B, de C y lo mismo con 'data2':

In [16]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [18]:
df.groupby('key').mean()

,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


Para saber cuál es la **media** de **cada columna del DataFrame** NO podria hacer `df.mean()` porque sale ERROR > ya que debo eliminar la columna 'key'para hacer el cálculo, porque es de tipo string entonces no puede calcular la media:

In [21]:
df.drop(columns='key').mean() # Con .drop() elimino dicha columna para el cálculo

data1    2.5
data2    4.5
dtype: float64

In [27]:
df['data1_media'] = df['data1'] - df['data1'].mean()
df

,key,data1,data2,data1_media
0,A,0,5,-2.5
1,B,1,0,-1.5
2,C,2,3,-0.5
3,A,3,3,0.5
4,B,4,7,1.5
5,C,5,9,2.5


> ### **El método `apply( )`**
Según Juan, es el método más útil de todos, se podría descartar el `.transform()` y utilizar `.apply()` en cambio.

Permite aplicar por ejemplo: 
- Una función `lambda`
- Una función que he creado. 

El método ``apply()`` permite aplicar una función arbitraria a los resultados del grupo.
La función debe tomar un ``DataFrame``, y devolver un objeto Pandas (por ejemplo, ``DataFrame``, ``Series``) o un escalar; la operación de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una ``apply()`` que **normaliza la primera columna por la suma de la segunda, es decir, divide la primer columna por la suma de la segunda SEGÚN LA AGRUPACIÓN y la transforma**:

In [28]:
def norm_by_data2(x):
    # x es un DataFrame de valores de grupo
    # x['data1'] = x['data1'] / x['data2'].sum()
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

df
  key  data1  data2  data1_media
0   A      0      5         -2.5
1   B      1      0         -1.5
2   C      2      3         -0.5
3   A      3      3          0.5
4   B      4      7          1.5
5   C      5      9          2.5

df.groupby('key').apply(norm_by_data2)
      key     data1  data2  data1_media
key                                    
A   0   A  0.000000      5         -2.5
    3   A  0.375000      3          0.5
B   1   B  0.142857      0         -1.5
    4   B  0.571429      7          1.5
C   2   C  0.166667      3         -0.5
    5   C  0.416667      9          2.5

In [29]:
df.groupby('key').sum()

,data1,data2,data1_media
key,,,
A,3,8,-2.0
B,5,7,0.0
C,7,12,2.0


In [30]:
print(3/8)

0.375


La función ``apply()`` dentro de un ``GroupBy`` es bastante flexible: el único criterio es que la función toma un ``DataFrame`` y devuelve un objeto Pandas o un escalar; ¡lo que hagas en medio depende de ti!

>> ### **Especificar la CLAVE de AGRUPACIÓN**

En los ejemplos sencillos presentados anteriormente, dividimos el ``DataFrame`` en un único nombre de columna.
Esta es sólo una de las muchas opciones por las que los grupos se pueden definir, y vamos a ir a través de algunas otras opciones para la especificación de grupo aquí.

> #### Una **LISTA, MATRIZ, SERIE O ÍNDICE** que proporcione las claves de agrupación.

- La CLAVE puede ser cualquier **serie o lista** cuya **longitud** **coincida con la del ``DataFrame``**. 

En el siguiente ejemplo:
- Estoy creando una lista 'L' donde paso una serie de números, y 
- Según la posición va a "reemplazar" a las key: A=0, B=1, C=0, A=1, B=2 y C=0. 
- Luego va a agrupar según esas claves, y en 'key' va a juntar las que correspondan según dicha agrupación.

In [31]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

,key,data1,data2,data1_media
0,A,0,5,-2.5
1,B,1,0,-1.5
2,C,2,3,-0.5
3,A,3,3,0.5
4,B,4,7,1.5
5,C,5,9,2.5
,key,data1,data2,data1_media
0,ACC,7,17,-0.5
1,BA,4,3,-1.0
2,B,4,7,1.5


Por supuesto, esto significa que hay otra forma más verbosa de realizar el ``df.groupby('clave')`` de antes:

In [ ]:
display('df', "df.groupby(df['key']).sum()")

> #### **Un DICCIONARIO, MAPPING o SERIE** que asigna un índice a un grupo

Otro método consiste en proporcionar un diccionario que asigne los valores del índice a las claves del grupo, mediante un **MAPEO**:

In [40]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2,data1_media
key,,,
A,0,5,-2.5
B,1,0,-1.5
C,2,3,-0.5
A,3,3,0.5
B,4,7,1.5
C,5,9,2.5
,data1,data2,data1_media
key,,,
consonant,12,19,2.0


> #### **Cualquier función Python**

De forma similar al mapeo, puede pasar cualquier función de Python que introduzca el valor del índice y genere el grupo.

En el siguiente ejemplo agrupo por las key y las convierto a minúsculas ('str.lower') > Esto serviría por si tengo un índice o columnas con las mismas letras pero algunas en mayúsculas y otras en minúsculas. 

In [ ]:
display('df2', 'df2.groupby(str.lower).mean()')

> #### **Una lista de claves válidas - MultiIndex** = Ejemplo: función de Python + mapping

Además, cualquiera de las opciones de clave anteriores puede combinarse para agruparse en un **MULTIÍNDICE** = Va a **agrupar según el orden que ponga las claves**. 

Ejemplos con distinto orden:

In [43]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2,data1_media
key,key,,,
a,vowel,1.5,4.0,-1.0
b,consonant,2.5,3.5,0.0
c,consonant,3.5,6.0,1.0


In [44]:
df2.groupby([mapping, str.lower]).mean()

data1  data2  data1_media
key       key                           
consonant b      2.5    3.5          0.0
          c      3.5    6.0          1.0
vowel     a      1.5    4.0         -1.0

>> ### Ejemplo de agrupación

Como ejemplo de esto, en un par de líneas de código Python podemos juntar todo esto y contar los planetas descubiertos por método y por década:

> CÓDIGO para calcular la DÉCADA según los años: 

`decade = 10 * (planets['year'] // 10)` <br>
Ejemplo: <br>
1989 // 10 = 198 (sin decimal, solo el entero)<br>
198 * 10 = 1980 > Da que sucedió en la década de los 80.

In [49]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


Esto demuestra el poder de la combinación de muchas de las operaciones que hemos discutido hasta ahora cuando se observan conjuntos de datos realistas.
Inmediatamente obtenemos una comprensión general de cuándo y cómo se han descubierto planetas en las últimas décadas.

Aquí sugeriría profundizar en estas pocas líneas de código, y evaluar los pasos individuales para asegurarse de que entiende exactamente lo que están haciendo al resultado.
Ciertamente es un ejemplo algo complicado, pero entender estas piezas te dará los medios para explorar de forma similar tus propios datos.